In [5]:
import os
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [4]:
!pip install langchain_community langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.3 MB/s eta 0:00:00


In [ ]:
   pdf_folder = "/content/Data"

In [ ]:
loader = PyPDFDirectoryLoader(pdf_folder)
documents = loader.load()

In [ ]:
#!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 15.9 MB/s eta 0:00:00


In [ ]:
print(f"Loaded {len(documents)} documents from {pdf_folder}")

Loaded 3219 documents from /content/Data


In [ ]:
# Split the documents into smaller chunks using RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)

# Print the first chunk to verify
print(final_documents[10916])

page_content='force for declaring any marriage to be null and void  or for annulling or dissolving any marriage or for 
judicial separation pending at the commencement of this Act, and any su ch proceeding may be continued 
and determined as if this Act had not been passed. 
(4) Nothing contained in this Act shall be deemed to affect the provisions contained in the Special 
Marriage Act, 1954, (43 of 1954) with respect to marriages between Hindus solemnized under that Act, 
whether before or after the commencement of this Act. 
30. [Repeals].—Rep. by the Repealing and Amending Act , 1960 (58 of 1960), s. 2 and the First 
Schedule (w.e.f. 26-12-1960). 
                                                           
1. Ins. by Act 49 of 2001, s. 9 (w.e.f. 24-9-2001). 
2. Subs. by Act 68 of 1976, s. 19 (w.e.f. 27-5-1976). 
3. Subs. by Act 50 of 2003, s. 5, for “period of thirty days” (w.e.f. 23-12-2003).' metadata={'source': '/content/Data/4.pdf', 'page': 12, 'page_label': '13'}


In [ ]:
len(final_documents)

10917

In [8]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

<ipython-input-8-97d0194c0452>:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings=HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-0.01379669 -0.00728636  0.00788882 -0.02068515  0.02761134 -0.00727931
  0.10330283 -0.05076543  0.02217055 -0.00922084 -0.00815161  0.01297491
 -0.00816634 -0.02459426 -0.01720368 -0.02949937 -0.03762061 -0.0011015
 -0.01232414  0.02537816  0.07706203  0.04522991 -0.02199267  0.00355094
  0.00620816  0.0454981   0.02263525 -0.0450823  -0.02221864 -0.14260259
 -0.03883087  0.02289567  0.04037417  0.00457559 -0.10627172 -0.03922089
 -0.06405871  0.01471103  0.02442922  0.03225086 -0.01709476  0.01960495
  0.03239483  0.02448255 -0.03381703  0.00673098  0.01091547  0.04232851
 -0.07833852 -0.0858651  -0.03782303 -0.03892377 -0.00318001  0.08809873
 -0.00639968 -0.07059135  0.04065747  0.025867    0.02476463  0.03201234
  0.03598166  0.00062948 -0.16298617  0.04412056  0.02751745 -0.01228786
  0.04688871  0.01634219  0.0334128   0.09206836 -0.04433559  0.05645799
 -0.0245967   0.00999324 -0.02207954 -0.03024874 -0.00235448  0.00199659
 -0.00561718  0.02798774 -0.01433944  0.00843264  0.

In [ ]:

vectorstore = FAISS.from_documents(final_documents, huggingface_embeddings)

In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.1 MB/s eta 0:00:00


In [33]:
output_dir = "/content/faiss"
vectorstore = FAISS.load_local(output_dir, huggingface_embeddings, allow_dangerous_deserialization=True)

In [34]:
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7906c2c84410>, search_kwargs={})

In [ ]:
'''
# Define the output directory to save the FAISS vector store
output_dir = "/content/output_faiss"  # Change this path as needed

# Save the FAISS vector database
vectorstore.save_local(output_dir)

print(f"Vector database saved to: {output_dir}")


Vector database saved to: /content/output_faiss


In [ ]:
'''
query="What are the provisions of the IPC regarding theft, robbery, and dacoity?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

section 395, IPC, 1860, and not under section 396.
Leading case:—Shyam Behari.
Preparation to commit dacoity is punishable (section 399). and so is either belonging
to a gang of dacoits (section 400), or assembling for the purpose of committing
dacoity (section 402).
Aggravated forms of robbery and dacoity are—
(1) Offender using any deadly weapon at the time of committing robbery or dacoity or
causing or attempting to cause death or grievous hurt to any person (section 397).
For the purpose of this section it is not necessary that the weapon should be actually
used. Mere carrying of the weapon causes a psychological sense of insecurity and fear
and this would constitute enough use within the meaning of this section.
Leading case:—Phool Kumar.


In [ ]:
#retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
#print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7bdff48c7290> search_kwargs={'k': 3}


In [35]:
from langchain_community.utilities import GoogleSerperAPIWrapper
serper = GoogleSerperAPIWrapper(serper_api_key="")

In [45]:
from langchain_core.tools import tool
@tool
def vector_search(query: str) -> str:
    """Search legal docs with conversation context"""
    # Automatically uses history-aware retriever
    docs = history_aware_retriever.invoke({
        "input": query,
        "chat_history": []  # Will be populated automatically
    })
    return "\n\n".join([d.page_content for d in docs][:3])  # Top 3 results

@tool
def web_search(query: str) -> str:
    """Search web using Google Serper API"""
    return GoogleSerperAPIWrapper().run(query)

tools = [vector_search, web_search]

In [37]:
llm=ChatGroq(groq_api_key="",
         model_name="deepseek-r1-distill-llama-70b")

In [38]:
from langchain_core.prompts import MessagesPlaceholder
retriever_prompt = """Given chat history and input, rephrase for standalone search:"""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", retriever_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [61]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a legal assistant. Use these steps:
1. First search local laws using vector_search
2. Then verify with web_search
3. Combine both sources

Follow these rules:
1. ALWAYS interpret pronouns ("their"/"they") using the MOST RECENT subject from chat history
2. If context changes (e.g., IPC → Bankruptcy), explicitly acknowledge the shift
3. Prioritize police roles unless explicitly stated otherwise"""),  # Critical addition
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])




In [40]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


In [41]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typin

In [48]:
class AutoContextSession:
    def __init__(self):
        self.history = []

    def ask(self, question):
        result = agent_executor.invoke({
            "input": question,
            "chat_history": self.history
        })

        # Store context automatically
        self.history.extend([
            HumanMessage(content=question),
            AIMessage(content=result['output'])
        ])

        return result['output']

session = AutoContextSession()


In [63]:
response1 = session.ask("What are the essential elements of criminal conspiracy under IPC?")
print("IPC Response:", response1)



> Entering new AgentExecutor chain...
The essential elements of criminal conspiracy under the Indian Penal Code (IPC) are defined under **Section 120A** and **Section 120B**. Here's a breakdown:

### **1. Agreement to Commit an Illegal Act**
   - **Section 120A**: Criminal conspiracy is defined as an agreement between two or more persons to do an illegal act or a legal act by illegal means.
   - **Key Element**: The primary element is the agreement or understanding between the parties to achieve an unlawful purpose.

### **2. Overt Act in Furtherance of the Conspiracy**
   - While not explicitly mentioned in the IPC, courts have interpreted that for a conspiracy to be actionable, there must be an overt act or some positive steps taken in furtherance of the conspiracy.
   - However, under the IPC, the mere agreement itself is sufficient to constitute conspiracy if the object of the agreement is unlawful.

### **3. Punishment**
   - **Section 120B**: The punishment for criminal conspir

In [54]:
response2 = session.ask("Can you provide an example of a situation where an act qualifies as causing death by negligence but does not amount to culpable homicide?")
print("\nFollow-up Response:", response2)



> Entering new AgentExecutor chain...
Certainly! Here's an example:

**Situation:**

A driver is operating a vehicle at a reasonable speed and is not under the influence of alcohol or drugs. However, the driver fails to notice a pedestrian stepping onto a crosswalk due to being distracted by adjusting the car's radio. As a result, the driver does not stop in time and strikes the pedestrian, causing fatal injuries.

**Analysis:**

- **Causing Death by Negligence:** The driver's failure to notice the pedestrian due to being distracted by adjusting the radio constitutes negligence. The driver had a duty to operate the vehicle safely and failed to do so by being distracted, which led to the pedestrian's death.

- **Not Culpable Homicide:** Culpable homicide typically involves a higher level of culpability, such as recklessness or intent. In this case, the driver did not act with reckless disregard for human life or with the intent to cause harm. The distraction was momentary and not indi